[View in Colaboratory](https://colab.research.google.com/github/wonseokjung/hands_on_tf/blob/master/2_Tensorflow_models.ipynb)

# Models & Layers 



1. We usually implement models as classes and use statements like y_pred( ) = model( x ) to call the models. 

2. The structure of a model class is rather simple which basically include __init__( ) ( for construction and initialization ) 

3. call(input ) while you can also define your own methods if necessary 




In [0]:
import tensorflow as tf

import numpy as np

tf.enable_eager_execution()

In [0]:


class MyModel(tf.keras.Model):
  
  def __init__(self):
    super().__init__()
    # Call method에서 불려질 initialization code를 넣는다. 
    
  def call(self, inputs): 
    
    #여기에는 모델 코드를 넣는다. 
    
    return output

Our model inherits from tf.kears.Model.

Keras is an advanced neural network API written in Python and is supperted by TensorFlow officially. 

One benefit of inheriting from tf.keras.Model is that we will be able to use several methods and attributes like acquiring all variables in the model through the model.variable attribute after the class is instantiated.




**y_pred = tf.matmul(X,w) + b  can be implemented thorugh model classes **

# EXAMPLE : Mnist , Multilayer Perceptron 



<img src="https://www.dropbox.com/s/afcyulmasb6cpp0/Screenshot%202018-09-30%2021.19.56.png?raw=1">
**bold text**

In [0]:
class DataLoader():
  def __init__(self):
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    
    self.train_data = mnist.train.images 
    self.train_labels = np.asarray(mnist.train.labels, dtype= np.int32)
    self.eval_data = mnist.test.images 
    self.eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
   

  def get_batch(self, batch_size):
    index = np.random.randint(0,np.shape(self.train_data)[0], batch_size)
    
    return self.train_data[index, :], self.train_labels[index]
  
  


In [0]:
class MLP(tf.keras.Model): 
  def __init__(self):
    super().__init__()
    
    self.dense1 = tf.keras.layers.Dense(units = 100, activation =tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(units = 10)
    
    
    
  def call(self, inputs):
    
    x = self.dense1(inputs)
    x = self.dense2(x)
    
    return x
    
    
  def predict(self, inputs ):
    
    logits = self(inputs)
    
    return tf.argmax(logits, axis =-1)
  
  
    
    

## Define hyperparameters 


In [0]:
num_batches = 1000

batch_size = 50

learning_rate = 0.001 

In [0]:
model = MLP()

In [11]:
data_loader = DataLoader() 

optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


### Iteration steps: 

1. DataLoader 를 통하여 training data의 set을  임의로 읽는다. 

2. data를 model로 feed한다. 

3. 정답과 prediction을 비교한다. ( loss function )

4. Loss function을 Differentiate한다. 

5. Loss를 Minimize하기위해 model parameter을 업데이트한다. 



In [21]:
for batch_index in range(num_batches):
  X,y =data_loader.get_batch(batch_size)
  
  with tf.GradientTape() as tape: 
    y_logit_pred = model(tf.convert_to_tensor(X))
    loss = tf.losses.sparse_softmax_cross_entropy(labels= y, logits = y_logit_pred)
    
    print(batch_index,loss.numpy())
    
  grads = tape.gradient(loss, model.variables)
  
  optimizer.apply_gradients(grads_and_vars= zip(grads, model.variables))

0 2.3836544
1 2.3577263
2 2.369278
3 2.132346
4 2.0906208
5 2.0831938
6 1.9190334
7 1.9203488
8 1.882431
9 1.9033542
10 1.7308813
11 1.6847018
12 1.7609009
13 1.5581069
14 1.5830148
15 1.5604353
16 1.4125894
17 1.4222411
18 1.3887985
19 1.2715633
20 1.2639377
21 1.1925238
22 1.2894301
23 1.0862559
24 1.177566
25 0.95479125
26 1.0635705
27 0.90716416
28 1.073618
29 0.9138944
30 0.89936185
31 1.10321
32 0.76090544
33 0.78239065
34 1.0342768
35 0.86361104
36 0.97453135
37 0.767191
38 0.7842591
39 0.6515729
40 0.94250923
41 0.84330106
42 0.82899356
43 0.8824417
44 0.71149206
45 0.4530648
46 0.693158
47 0.6707597
48 0.79366845
49 0.7266124
50 0.7087634
51 0.7537384
52 0.6567216
53 0.53287256
54 0.60967433
55 0.6544329
56 0.5755513
57 0.5667496
58 0.5499377
59 0.43579963
60 0.56829375
61 0.4802512
62 0.64409304
63 0.46271706
64 0.55420303
65 0.46038255
66 0.5470114
67 0.6220205
68 0.56924814
69 0.4529014
70 0.502906
71 0.4191805
72 0.5958595
73 0.53185385
74 0.5823629
75 0.5250027
76 0.62656

In [23]:
num_eval_samples = np.shape(data_loader.eval_labels)[0]

y_pred = model.predict(data_loader.eval_data).numpy() 

print("accuracy:%f"%(sum(y_pred == data_loader.eval_labels) / num_eval_samples))

accuracy:0.948800
